# A Psi4 cheat sheet

For other examples see: [Andrea Snow's psi4 cheat sheet](https://adreasnow.com/Cheat%20Sheets%20and%20Play/Cheat%20Sheets/Psi4/)

---

## Input
### Importing `psi4` and `fortecubeview`

`fortecubeview` provides several helpful functionality missing from psi4.

In [ ]:
import psi4
import fortecubeview

### Defining a molecule using Cartesian coordinates (units of Å)

In [ ]:
mol = psi4.geometry("""
H 0.0 0.0 0.0
H 0.0 0.0 0.75
""")

### Defining a molecule using Z-matrix coordinates (units of Å)

In [ ]:
mol = psi4.geometry("""
H
H 1 0.75
""")

### Get the geometry from PubChem

Always check the geometry before using it!

In [ ]:
methanol = psi4.geometry("""
pubchem:methanol""")

fortecubeview.geom(molecule=methanol)

### Getting the coordinates from a molecule object

In [ ]:
print(mol.to_string(dtype='xyz'))

## Computations

### Setting the output file

In [ ]:
psi4.set_output_file('output.txt',False)

### Hartree–Fock energy (closed shell)
The string `SCF/cc-pVDZ` specifies a self-consistent-field computation using the cc-pVDZ basis set

In [ ]:
psi4.energy('SCF/cc-pVDZ')

### Useful SCF options
- `MAXITER`: maximum number of iterations
- `E_CONVERGENCE`: energy convergence criterion
- `D_CONVERGENCE`: density matrix convergence criterion
- `REFERENCE`: any of `RHF`, `ROHF`, `UHF`
- `GUESS`: orbital guess
- `SCF_TYPE`: `DF` (default), `DIRECT`, `PK`

Note how this energy differs from the one above (due to the use of exact two-electron integrals instead of density fitted ones)

In [ ]:
psi4.set_options({'E_CONVERGENCE' : 12,
                  'D_CONVERGENCE' : 6,
                  'SCF_TYPE' : 'PK',
                  'GUESS' : 'SAD'
                 })

psi4.energy('SCF/cc-pVDZ')

### Showing the MO coefficients
Set the `PRINT_MOS` option to `True` to print the MO coefficient matrix at the end of a computation.

In [ ]:
mol = psi4.geometry("""
H
H 1 0.75
""")

psi4.set_options({'PRINT_MOS' : True})
psi4.energy('SCF/cc-pVDZ')
!cat output.txt | tail -48 | head -36

### Unrestricted HF computation

Set the keyword `REFERENCE` to `UHF` 

In [ ]:
# O2 triplet 
# (charge = 0) (multiplicity = 3)
mol = psi4.geometry("""
0 3 
O
O 1 1.208
""")

psi4.set_output_file('o2trip_uhf_cc-pvdz.txt',False)
psi4.set_options({'REFERENCE' : 'UHF'})
psi4.energy('hf/cc-pVDZ')

### Restricted open-shell HF computation

Set the keyword `REFERENCE` to `ROHF` 

In [ ]:
# O2 triplet 
# (charge = 0) (multiplicity = 3)
mol = psi4.geometry("""
0 3 
O
O 1 1.208
""")

psi4.set_output_file('o2trip_rohf_cc-pvdz.txt',False)
psi4.set_options({'REFERENCE' : 'ROHF'})
psi4.energy('hf/cc-pVDZ')

### Plotting orbitals

Best practices
- save the orbitals to a separate directory (`!mkdir oh_mos`, `'CUBEPROP_FILEPATH' : 'oh_mos'`)
- cube files can take a lot of hard drive space. Specify which orbitals to generate (`'CUBEPROP_ORBITALS' : [1,2,...,-1,-2,...]`). Psi4 uses a `-` to indicate beta orbitals

In [ ]:
# Part I: Make cube files

# doublet OH UHF orbitals
mol = psi4.geometry("""
0 2
H
O 1 1.3
""")

# create the folder oh_mos/
!mkdir oh_mos

psi4.set_options({'REFERENCE' : 'UHF',
                  'CUBEPROP_FILEPATH' : 'oh_mos',
                  'CUBEPROP_TASKS' : ['ORBITALS','DENSITY'],
                  'CUBEPROP_ORBITALS' : [1,2,3,4,5,6,7,8,9,-1,-2,-3,-4,-5,-6,-7,-8,-9]})

# run computation
E, wfn = psi4.energy('hf/cc-pVDZ', return_wfn=True)

# generate cube files
psi4.cubeprop(wfn)

In [ ]:
# Part II: Visualize cube files in the folder oh_mos/ with fortecubeview (alternatively use vmd and vmd_cube)

import fortecubeview
fortecubeview.plot('oh_mos')

### HF geometry optimization
Notice that the `mol` object is update with the new geometry

In [ ]:
mol = psi4.geometry("""
H 0.0 0.0 0.0
H 0.0 0.0 0.75
""")

psi4.optimize('SCF/cc-pVDZ')
print(mol.to_string(dtype='xyz'))

### HF frequency calculation

**You MUST optimize the geometry before running frequencies at the same level of theory**

In [ ]:
mol = psi4.geometry("""
H 0.0 0.0 0.0
H 0.0 0.0 0.75
""")

psi4.set_output_file('h2_freq.txt',False)

# run an optimization
psi4.optimize('SCF/cc-pVDZ')
print(mol.to_string(dtype='xyz'))

# run a frequency computation
psi4.frequencies('SCF/cc-pVDZ')

!cat h2_freq.txt | tail -95 | head -33

### Visualize vibrational modes

Add the options to write a normal mode file
```python
psi4.set_options({'NORMAL_MODES_WRITE' : True,
                  'WRITER_FILE_LABEL' : 'normal_modes'})
```

The line
```python
normalmode_file = psi4.core.get_writer_file_prefix(mol.name()) + ".molden_normal_modes"
```
automatically finds the name of the normal mode file

In [ ]:
# Part I: Compute the normal modes and write a normal mode file

mol = psi4.geometry("""
O
H 1 0.9
H 1 0.9 2 104.5
""")

psi4.set_output_file('h2o_freq.txt',False)

psi4.set_options({'NORMAL_MODES_WRITE' : True,
                  'WRITER_FILE_LABEL' : 'normal_modes'})

psi4.optimize('SCF/cc-pVDZ')
psi4.frequencies('SCF/cc-pVDZ')

# this is a hack
normalmode_file = psi4.core.get_writer_file_prefix(mol.name()) + ".molden_normal_modes"

In [ ]:
# Part II: Visualize normal mode file with fortecubeview (alternatively use molden or a compatible software)

fortecubeview.vib(normalmode_file)